In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from sqlalchemy import create_engine
import pymysql
pymysql.install_as_MySQLdb()
import os
import sys, urllib, json

In [ ]:
class soda_data_util:
    def __init__(self, file_path, encoding='utf-8'):
        print('========================================开始读取数据========================================')
        # 读取csv表格
        self.csv_data = pd.read_csv(file_path, 
                        encoding = encoding, 
                        header=None, 
                        names=['card_id', 'date', 'time', 'line', 'transport', 'amount', 'is_discount'])
        print('==================================数据读取完毕，数据的shape为:'+str(self.csv_data.shape)+"==================================")

In [ ]:
host = '127.0.0.1'
port = 3306
db = 'soda_data_schema'
user = 'root'
password = '19940301'
engine = create_engine(str(r"mysql+mysqldb://%s:" + '%s' + "@%s/%s") % (user, password, host, db))
transport_dict = {'P+R停车场':0, '公交':1, '出租':2, '地铁':3, '轮渡':4}
is_discount = {'优惠':0, '非优惠':1, '无':2}

In [ ]:
csv_list = []
def gci(filepath):
#遍历filepath下所有文件，包括子目录
    files = os.listdir(filepath)
    for fi in files:
        fi_d = os.path.join(filepath,fi)
        if os.path.isdir(fi_d):
            gci(fi_d)
        else:
            file_path = os.path.join(filepath,fi_d)
            if file_path.endswith('csv'):
                #print(file_path)
                csv_list.append(file_path)
    return csv_list
csv_list = gci('/usr/soda_data_set/gonggongjiaotongkagufenyouxiangongsi')
print(len(csv_list))

In [ ]:
import time
for i in range(0,1):
    start_time = time.time()
    print('这一次的第一个文件序号：' + str(i))
    df = soda_data_util(file_path= csv_list[i], encoding= 'GB2312').csv_data
    for j in range(i*3 + 1, i*3 + 3):
        print('这个文件序号拼接的文件是： ' + str(j))
        df = df.append(soda_data_util(file_path= csv_list[j], encoding= 'GB2312').csv_data)
    #print(df)
    df['date_time'] = df['date'] + ' ' + df['time']
    df['transport'] = df['transport'].apply(lambda x: transport_dict[x])
    df['is_discount'] = df['is_discount'].apply(lambda x: is_discount[x])
    df['week'] = df['date_time'].apply(lambda x: datetime.strptime(x,'%Y-%m-%d %H:%M:%S').strftime("%w"))
    df = df.drop(['date', 'time'], axis=1)
    print(df.shape)
    print(df.head(10))
    end_time = time.time()
    print(end_time - start_time)
    df.to_sql('yikatong_tb', con=engine, if_exists='append', index=False)
    end_time = time.time()
    print(end_time - start_time)

In [ ]:
import time
for csv_path in csv_list[0:1]:
    start_time = time.time()
    print(csv_path)
    df = soda_data_util(file_path= csv_path, encoding= 'GB2312').csv_data
    #print(df)
    df['date_time'] = df['date'] + ' ' + df['time']
    df['transport'] = df['transport'].apply(lambda x: transport_dict[x])
    df['is_discount'] = df['is_discount'].apply(lambda x: is_discount[x])
    df['week'] = df['date_time'].apply(lambda x: datetime.strptime(x,'%Y-%m-%d %H:%M:%S').strftime("%w"))
    df = df.drop(['date', 'time'], axis=1)
    print(df.head(10))
    df.to_sql('yikatong_tb', con=engine, if_exists='append', index=False)
    end_time = time.time()
    print(end_time - start_time)